In [1]:
# Imports and Setup
import sys
from pathlib import Path
import pandas as pd

# Ensure we are operating relative to the /functions directory
FUNCTIONS_DIR = Path.cwd().parent
sys.path.insert(0, str(FUNCTIONS_DIR))
print(f'FUNCTIONS_DIR: {FUNCTIONS_DIR}')

# Firebase setup
from firebase_admin import initialize_app, credentials, get_app
from google.cloud import firestore as google_firestore
import logging

logging.basicConfig(level=logging.INFO)

# Use direct path to service account key inside /functions
service_account_path = FUNCTIONS_DIR / 'serviceAccountKey.json'

# Initialize Firebase Admin SDK
try:
    app = get_app()
except ValueError:
    cred = credentials.Certificate(service_account_path)
    app = initialize_app(cred, {'storageBucket': 'uno-y-b48fb.appspot.com'})

# Create Firestore client using the same credentials
firestore_client = google_firestore.Client.from_service_account_json(str(service_account_path))

# planogram imports
from planogram import data_loader

# Load data
print("Loading data...")
firebase_data = data_loader.load_firebase_collections(firestore_client)  # Loads all collections
sales_data, latest_sales_df = data_loader.load_all_sales_data(firestore_client, fetch_latest_sales=True, firebase_data=firebase_data) # Loads and combines historical + latest sales
#sales_data = data_loader.enrich_with_purchase_prices(sales_data, firebase_data.get('product_purchase_prices'))
print("Data loaded.")


FUNCTIONS_DIR: /Users/sam/Documents/Code/Planogram/uno/functions


Loading data...
Loading collection: app_machines
app_machines slots cleaned: 8193 -> 8172 (removed 21, 0.26%)
app_machines: 234 rows (machines themselves not filtered)
Loading collection: products
products (filtered unspecific rows): 2410 -> 1961 rows (removed 449, 18.63%)
Loading collection: product_purchase_prices
product_purchase_prices: 1370 -> 1370 rows (removed 0, 0.00%)
Loading collection: product_nayax_mapping
product_nayax_mapping: 5167 -> 5167 rows (removed 0, 0.00%)
Loading historical sales from Parquet files...
Historical sales (after filtering): 6979402 -> 5390446 rows (removed 1588956, 22.77%)
Fetching latest sales data...
Fetching sales after: 2025-11-18 23:59:50+00:00
Enriching latest sales data...
Cached machines data missing 'address' column (likely app_machines). Fetching full machines data...
Fetching machines collection...


/Users/sam/Documents/Code/Planogram/uno/functions/planogram/data_loader.py:345: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  enriched_df[col] = enriched_df[col].fillna(False).astype(bool)
/Users/sam/Documents/Code/Planogram/uno/functions/planogram/data_loader.py:345: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  enriched_df[col] = enriched_df[col].fillna(False).astype(bool)


Data loaded.


In [2]:
import pandas as pd

def check_dtype_mismatch(df1, df2, df1_name="Historical", df2_name="Latest"):
    """
    Checks for column data type mismatches between two DataFrames.
    """
    print(f"🔍 DTYPE MISMATCH CHECK: {df1_name} vs {df2_name}")
    print("-" * 60)
    
    common_cols = set(df1.columns) & set(df2.columns)
    mismatches = 0
    
    for col in sorted(common_cols):
        dtype1 = df1[col].dtype
        dtype2 = df2[col].dtype
        
        # Handle nullable types vs numpy types (e.g., Int64 vs int64, float64 vs Float64)
        # We consider them compatible if they represent the same kind of data
        is_compatible = False
        
        if dtype1 == dtype2:
            is_compatible = True
        elif pd.api.types.is_integer_dtype(dtype1) and pd.api.types.is_integer_dtype(dtype2):
            is_compatible = True
        elif pd.api.types.is_float_dtype(dtype1) and pd.api.types.is_float_dtype(dtype2):
            is_compatible = True
        elif pd.api.types.is_string_dtype(dtype1) and pd.api.types.is_string_dtype(dtype2):
            is_compatible = True
        elif pd.api.types.is_object_dtype(dtype1) and pd.api.types.is_string_dtype(dtype2):
             # Object often contains strings in pandas
            is_compatible = True
        elif pd.api.types.is_string_dtype(dtype1) and pd.api.types.is_object_dtype(dtype2):
            is_compatible = True
            
        if not is_compatible:
            print(f"❌ Mismatch in '{col}':")
            print(f"   {df1_name}: {dtype1}")
            print(f"   {df2_name}: {dtype2}")
            mismatches += 1

    if mismatches == 0:
        print("✅ No significant data type mismatches found.")
    else:
        print(f"\n⚠️ Found {mismatches} column(s) with mismatched types.")

# Usage:
check_dtype_mismatch(sales_data, latest_sales_df)

🔍 DTYPE MISMATCH CHECK: Historical vs Latest
------------------------------------------------------------
❌ Mismatch in 'is_ICA_refiller':
   Historical: object
   Latest: bool

⚠️ Found 1 column(s) with mismatched types.


In [3]:
import pandas as pd
import numpy as np

def evaluate_data_mapping(latest_df, historical_df):
    print("="*50)
    print("DATA MAPPING EVALUATION REPORT")
    print("="*50)

    if latest_df.empty:
        print("❌ Latest sales DataFrame is empty! Nothing to evaluate.")
        return

    # 1. Basic Stats
    print(f"📊 New Records Loaded: {len(latest_df):,}")
    print(f"📅 Date Range: {latest_df['local_timestamp'].min()} to {latest_df['local_timestamp'].max()}")
    
    # Check continuity with historical data
    if not historical_df.empty:
        last_hist_date = historical_df['local_timestamp'].max()
        print(f"🔄 Historical Data Ends: {last_hist_date}")
        if latest_df['local_timestamp'].min() > last_hist_date:
            print("✅ Data is continuous (New starts after Old)")
        else:
            print(f"⚠️ Overlap detected! {len(latest_df[latest_df['local_timestamp'] <= last_hist_date])} records overlap.")

    print("\n" + "-"*50)
    print("🧩 ENRICHMENT SUCCESS RATES")
    print("-" * 50)

    # 2. Product Enrichment (Mapping nayax_name -> product_name)
    total_rows = len(latest_df)
    mapped_products = latest_df['product_name'].notna().sum()
    pct_products = (mapped_products / total_rows) * 100
    
    print(f"📦 Product Mapping: {pct_products:.1f}% ({mapped_products}/{total_rows})")
    
    if pct_products < 100:
        missing_products = latest_df[latest_df['product_name'].isna()]['nayax_name'].unique()
        print(f"   ⚠️ {len(missing_products)} unique 'nayax_name' values failed to map.")
        print(f"   Top 5 unmapped: {list(missing_products[:5])}")

    # 3. Machine Enrichment (Mapping machine_key -> address/machine_model)
    # Using 'address' as a proxy for successful machine enrichment
    mapped_machines = latest_df['address'].notna().sum()
    pct_machines = (mapped_machines / total_rows) * 100
    
    print(f"🤖 Machine Mapping: {pct_machines:.1f}% ({mapped_machines}/{total_rows})")
    
    if pct_machines < 100:
        missing_machines = latest_df[latest_df['address'].isna()]['machine_key'].unique()
        print(f"   ⚠️ {len(missing_machines)} unique 'machine_key' values failed to map.")
        print(f"   Top 5 unmapped keys: {list(missing_machines[:5])}")

    print("\n" + "-"*50)
    print("🔍 COLUMN CONSISTENCY")
    print("-" * 50)

    # 4. Column Comparison
    hist_cols = set(historical_df.columns)
    new_cols = set(latest_df.columns)
    
    missing_in_new = hist_cols - new_cols
    extra_in_new = new_cols - hist_cols
    
    if not missing_in_new and not extra_in_new:
        print("✅ Columns match perfectly between Historical and Latest.")
    else:
        if missing_in_new:
            print(f"❌ Missing columns in Latest: {missing_in_new}")
        if extra_in_new:
            print(f"ℹ️ Extra columns in Latest: {extra_in_new}")

    # 5. Null Value Check for Critical Columns
    print("\n" + "-"*50)
    print("🛑 NULL VALUE CHECK (Critical Columns)")
    print("-" * 50)
    critical_cols = ['category', 'ean', 'purchase_price_kr', 'machine_group_tag', 'refiller']
    
    print(f"{'Column':<20} | {'Null Count':<10} | {'Null %':<10}")
    print("-" * 46)
    for col in critical_cols:
        if col in latest_df.columns:
            null_count = latest_df[col].isna().sum()
            null_pct = (null_count / total_rows) * 100
            print(f"{col:<20} | {null_count:<10} | {null_pct:<9.1f}%")
        else:
            print(f"{col:<20} | {'MISSING':<10} | -")

# Run the evaluation
evaluate_data_mapping(latest_sales_df, sales_data)

DATA MAPPING EVALUATION REPORT
📊 New Records Loaded: 148,502
📅 Date Range: 2025-11-19 00:00:00+00:00 to 2025-12-11 00:00:00+00:00
🔄 Historical Data Ends: 2025-11-18 23:59:50+00:00
✅ Data is continuous (New starts after Old)

--------------------------------------------------
🧩 ENRICHMENT SUCCESS RATES
--------------------------------------------------
📦 Product Mapping: 99.2% (147298/148502)
   ⚠️ 211 unique 'nayax_name' values failed to map.
   Top 5 unmapped: ['Powerking Proteinbar Choklad', 'Protein Smoothie', 'Dr Pepper 33cl', 'Mama Chin Curry Kyckling', 'Sätra Fralla Kalkon&ost']
🤖 Machine Mapping: 78.2% (116148/148502)
   ⚠️ 165 unique 'machine_key' values failed to map.
   Top 5 unmapped keys: ['Go Energy - Padel Enköping_268966842', 'Go Energy - SÖS 1_450007630', 'Pro Ny - Södertälje Akuten_418508875', 'Energi - Dalstorp padel _341395781', 'Go Energy -  SÖS 3_450007629']

--------------------------------------------------
🔍 COLUMN CONSISTENCY
-----------------------------------